# This file runs study 3, varying best values, without spillover effects

In [1]:
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from numpy.core.fromnumeric import repeat
from scipy import stats
import seaborn as sns
import pandas as pd
import scipy.stats as st

In [2]:
class MAB:
    def __init__(self, probs, karms, nfirst, nEpisodes):
        '''
        meanings for the parameters:
        probs: True success rate for each arm k in {1, ..., K} 
        kArms: Number of arms to choose among
        nsamples: Number of subjects to test at each time step

        '''
        self.n_obs = 4000
        self.probs = probs
        self.K = karms
        self.n = int((self.n_obs-nfirst)/(nEpisodes-1)) if nEpisodes > 1 else 0
        self.T = nEpisodes
        self.first = nfirst
        self.true_win_arm = np.argmax(np.asarray(probs))

    def best_arm(self, s:np.ndarray, asgn):
        draws = 10000
        # + 1 for a Beta(1, 1) prior
        new_alpha = s+1
        new_beta = asgn-s+1
        selection_table = []
        for i in range(self.K):
            theta = np.random.beta(new_alpha[i], new_beta[i], draws)
            selection_table.append(theta)

        winning_arms = np.argmax(selection_table, axis = 0)
        winning_prob = []
        
#         for i in range(self.K):
#             winning_prob.append(winning_arms.count(i)/draws)
        c = Counter(winning_arms)
        winning_prob = np.zeros(self.K)
        for i in range(self.K):
            winning_prob[i] = c[i]/draws

        return winning_prob


    def reset(self):
        # initialization
        success = np.zeros((self.T, self.K))   # record the success cases of each arm at each time step
        assigned = np.zeros((self.T, self.K)) # record the assigned cases of each arm at each time step
        posterior_probs = 0 # estimated success rate
        winning_probs = np.zeros((self.T, self.K)) # probability to be the true best arm
        # rewards = np.zeros(self.T) # record the average rewards per period
        # regrets = np.zeros(self.T) # record the regrets per period
        return (success, assigned, posterior_probs, winning_probs)


    def do_experiment(self, assignments):
        c = Counter(assignments)
        assigned = [c[i] for i in range(self.K)]
        success = [np.random.binomial(assigned[i], self.probs[i]) for i in range(self.K)]
        return assigned, success


    def Thompson_sampling(self):
        # initialization
        (success, assigned, posterior_probs, winning_probs) = self.reset()

        # for round 1, samples are randomly assigned
        new_assign = np.random.choice(range(self.K), size=self.first)
        assigned[0], success[0]  = self.do_experiment(new_assign)
        posterior_probs = success[0,0]/ assigned[0,0]
        winning_probs[0] = self.best_arm(success[0], assigned[0])
        adj_probs = success[0][0]*self.K/self.first
        # print(adj_probs)
        if self.T > 1:
            count0=0
            for i in range(1, self.T):              
                new_assign = np.random.choice(range(self.K), size=self.n, p=list(winning_probs[i-1])) # adaptive design
                new_assigned, new_success = self.do_experiment(new_assign)
                assigned[i] = assigned[i-1]+ new_assigned
                success[i] = success[i-1]+ new_success
                winning_probs[i] = self.best_arm(success[i], assigned[i])
                if winning_probs[i-1,0] == 0:
                    count0 += 1
                else:
                    adj_probs += new_success[0]/winning_probs[i-1,0]/self.n
            posterior_probs = adj_probs / (self.T-count0)
        return (success, assigned, posterior_probs, winning_probs)
    


    def static(self):
        # initialization
        (success, assigned, posterior_probs, winning_probs) = self.reset()

        # for round 1, samples are randomly assigned
        new_assign = np.random.choice(range(self.K), size=self.first)
        assigned[0], success[0]  = self.do_experiment(new_assign)
        posterior_probs = success[0,0]/ assigned[0,0]
        winning_probs[0] = self.best_arm(success[0], assigned[0])
        adj_probs = success[0][0]*self.K/self.first
        for i in range(1,self.T):
            new_assign = np.random.choice(range(self.K), size=self.n) # static design
            new_assigned, new_success = self.do_experiment(new_assign)
            assigned[i] = assigned[i-1]+ new_assigned
            success[i] = success[i-1]+ new_success
            winning_probs[i] = self.best_arm(success[i], assigned[i])
            adj_probs += new_success[0]*self.K/self.n
        posterior_probs = adj_probs / self.T


        return (success, assigned, posterior_probs, winning_probs)
        
    def do_replication(self, times, method):
        '''
        parameters:
        final_regrets -- to record the final regret of each replication, in order to compare efficiency
        final_win_arm -- to record the final win arm selected by two methods, in order to compare the accurarcy
        final_win_probs -- to record the final probability of each arm being the best arm, in order to compare the accurarcy
        final_assignment -- to record the final number of assigned subjects to the true best arm, comparing the exploitition
        cum_rewards -- to record the total rewards of every replication
        '''
        # records of the replication
        final_win_arm = np.zeros(times)
        final_win_probs = np.zeros(shape = (times, self.K))
        final_assignment = np.zeros(times) # only record the true best arm
        estimation = np.zeros(times)
        
        if method == 'TS':
            for i in range(times):
                (success, assigned, posterior_probs, winning_probs) = self.Thompson_sampling()
                final_win_arm[i] = np.argmax(winning_probs[-1])
                final_win_probs[i] = winning_probs[-1]
                final_assignment[i] = assigned[-1][self.true_win_arm]
                estimation[i] = posterior_probs
        else:
            for i in range(times):
                (success, assigned, posterior_probs, winning_probs) = self.static()
                final_win_arm[i] = np.argmax(winning_probs[-1])
                final_win_probs[i] = winning_probs[-1]
                final_assignment[i] = assigned[-1][self.true_win_arm]
                estimation[i] = posterior_probs
        ate = np.mean(estimation)
        mse = np.mean(np.square(np.subtract(estimation,self.probs[0])))
        rmse = np.sqrt(mse)
        # rmse = np.std(estimation,0)
#         coverage = best_coverage(estimation, 5000,100,self.probs[0])
        low,high=st.t.interval(0.95, len(estimation)-1, loc=np.mean(estimation), scale=st.sem(estimation))
        win_counts = Counter(final_win_arm)
        estimate = {"best_selected":win_counts[0]/times,"ATE":ate, "RMSE":rmse, "Coverage":(low,high)}
        print(estimate)
        return (final_win_arm, final_win_probs, final_assignment, estimation)

In [3]:
# experiment 1
k = 9 # number of treatments
np.random.seed(99332)
first = 400
periods = 10
prob_other = [0.1]*8

In [4]:
# exp 1
probs = [0.1] + prob_other
sim1 = MAB(probs,k,first, periods)
(s1_ts_final_win_arm, s1_ts_final_win_probs, s1_ts_final_assignment, s1_ts_estimation) = sim1.do_replication(50, "TS")
(s1_st_final_win_arm, s1_st_final_win_probs, s1_st_final_assignment, s1_st_estimation) = sim1.do_replication(50, "static")

{'best_selected': 0.12, 'ATE': 0.11052344791607699, 'RMSE': 0.06593300722334038, 'Coverage': (0.09183790774652335, 0.12920898808563064)}
{'best_selected': 0.08, 'ATE': 0.09765, 'RMSE': 0.015609692501775938, 'Coverage': (0.09321980946819083, 0.10208019053180917)}


In [5]:
# exp 2
probs = [0.12] + prob_other
sim2 = MAB(probs,k,first, periods)
(s2_ts_final_win_arm, s2_ts_final_win_probs, s2_ts_final_assignment, s2_ts_estimation) = sim2.do_replication(5000, "TS")
(s2_st_final_win_arm, s2_st_final_win_probs, s2_st_final_assignment, s2_st_estimation) = sim2.do_replication(5000, "static")

{'best_selected': 0.5762, 'ATE': 0.12085822674129426, 'RMSE': 0.04353107028129781, 'Coverage': (0.11965145088634949, 0.12206500259623904)}
{'best_selected': 0.4796, 'ATE': 0.11972474999999999, 'RMSE': 0.01638306633387047, 'Coverage': (0.11927055162734032, 0.12017894837265966)}


In [6]:
# exp 3
probs = [0.14] + prob_other
sim3 = MAB(probs,k,first, periods)
(s3_ts_final_win_arm, s3_ts_final_win_probs, s3_ts_final_assignment, s3_ts_estimation) = sim3.do_replication(5000, "TS")
(s3_st_final_win_arm, s3_st_final_win_probs, s3_st_final_assignment, s3_st_estimation) = sim3.do_replication(5000, "static")

{'best_selected': 0.929, 'ATE': 0.13945265621446915, 'RMSE': 0.01868455256251843, 'Coverage': (0.1389348014600713, 0.13997051096886698)}
{'best_selected': 0.8342, 'ATE': 0.13986315, 'RMSE': 0.01783849173837295, 'Coverage': (0.13936854666747975, 0.14035775333252024)}


In [7]:
# exp 4
probs = [0.16] + prob_other
sim4 = MAB(probs,k,first, periods)
(s4_ts_final_win_arm, s4_ts_final_win_probs, s4_ts_final_assignment, s4_ts_estimation) = sim4.do_replication(5000, "TS")
(s4_st_final_win_arm, s4_st_final_win_probs, s4_st_final_assignment, s4_st_estimation) = sim4.do_replication(5000, "static")

{'best_selected': 0.9936, 'ATE': 0.16013405883698914, 'RMSE': 0.013539722056405485, 'Coverage': (0.15975865375496856, 0.16050946391900972)}
{'best_selected': 0.9776, 'ATE': 0.1596924, 'RMSE': 0.018882961234933465, 'Coverage': (0.1591688909919096, 0.16021590900809043)}


In [8]:
# exp 5
probs = [0.18] + prob_other
sim5 = MAB(probs,k,first, periods)
(s5_ts_final_win_arm, s5_ts_final_win_probs, s5_ts_final_assignment, s5_ts_estimation) = sim5.do_replication(5000, "TS")
(s5_st_final_win_arm, s5_st_final_win_probs, s5_st_final_assignment, s5_st_estimation) = sim5.do_replication(5000, "static")

{'best_selected': 0.9988, 'ATE': 0.17994460350517089, 'RMSE': 0.013055946494459326, 'Coverage': (0.17958259719640313, 0.18030660981393865)}
{'best_selected': 0.9976, 'ATE': 0.18024255000000003, 'RMSE': 0.020002866982010356, 'Coverage': (0.17968796003953338, 0.18079713996046667)}


In [9]:
# exp 6
probs = [0.22] + prob_other
sim6 = MAB(probs,k,first, periods)
(s6_ts_final_win_arm, s6_ts_final_win_probs, s6_ts_final_assignment, s6_ts_estimation) = sim6.do_replication(5000, "TS")
(s6_st_final_win_arm, s6_st_final_win_probs, s6_st_final_assignment, s6_st_estimation) = sim6.do_replication(5000, "static")

{'best_selected': 1.0, 'ATE': 0.21980907155615778, 'RMSE': 0.010649088518003252, 'Coverage': (0.21951384575470864, 0.22010429735760692)}
{'best_selected': 1.0, 'ATE': 0.2198205, 'RMSE': 0.022017415266102423, 'Coverage': (0.21921003103910433, 0.22043096896089567)}
